![](https://raw.githubusercontent.com/unm-escape/escape2022/main/header2.png)

<h1 style="text-align:center; color:#3333ff;">Plotting seismic reflection data and picking arrivals</h1>
<br>
<div style="text-align:center; font-size:16px">
    Earth and Planetary Sciences,<br>
    University of New Mexico<br>
    <br>
    August 17, 2022
</div>

---
**Tutorial Outline:**
1. Reading SEG-Y data from file<br>
2. Updating header info<br>
3. Applying simple amplitude attenuation correction<br>
4. Picking arrivals<br>
---
**Dependencies:** Obspy, Numpy, Matplotlib and the Utils script

---

<font size="4.5">**1. Reading SEG-Y data from file**</font><br>

Having looked at earthquake seismograms, the goal of this exercise is to quickly look at seismic reflection data while leveraging the Obspy and Matplotlib skills you acquired in previous lectures.

There are more robust python and non-python open-source frameworks for processing seismic reflection data. I recommend [Seismic Unix](https://wiki.seismic-unix.org/doku.php) and [Madagascar](https://reproducibility.org/wiki/Main_Page).

<font size="4.5">Importing required modules:</font><br>

The Utils module is a custom file containing several functions we will need to handle the SEG-Y files. Modules allow for breaking large programs into small manageable files and makes code reusable. 

Let's start by importing required modules.

In [ ]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
from obspy import Stream, read
from utils import plot_segy, pick_segy, plot_segy_picks, gain

Read the SEG-Y data into a [Stream object](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html#obspy.core.stream.Stream) by supplying obspy [read function](https://docs.obspy.org/packages/autogen/obspy.core.stream.read.html) with the path to the SEG-Y file on your computer.

The dataset we are using in this exercise, known as the OZ-25 dataset, is widely used in the geophysics community (see Yilmaz (2001), Chen et al. (2015) and Yarham et al. (2006)). The data is available at the [Seismic Unix website](https://wiki.seismic-unix.org/tutorials:data) and the [madagascar website](https://www.reproducibility.org/wiki/Main_Page).

In [ ]:
stream = read('./data/ozdata_25.segy', format='SEGY',check_compression=False,unpack_trace_headers=True)
print(stream)

Let's grab the first trace object and view a summary its metadata.

In [ ]:
tr = stream[0]
print(tr)

Let's view the actual trace data

In [ ]:
print(tr.data)

---
<font size="4.5">**2. Updating header info**</font><br>

The data is a common shot gather with a time sampling rate of 0.002s and receiver spacing of 0.05 km (50 m). The shot location is at the center of the array. 

Key headers of the OZ-25 data aren't populated, so let's update the receiver location, source location and the offset assuming a 2-D line.

In [ ]:
sx = 2350 # source x coordinate
gx = -0.05*1000 # Initialize receiver x coordinate at -50 m

# Loop throught the stream
for tr in stream:
    gx += 0.05*1000
    tr.stats.segy.trace_header.group_coordinate_x = gx
    tr.stats.segy.trace_header.source_coordinate_x = sx
    tr.stats.segy.trace_header.distance_from_center_of_the_source_point_to_the_center_of_the_receiver_group = gx-sx
print("Update done!")

---
<font size="4.5">**3. Applying simple amplitude attenuation correction**</font><br>

Seismic amplitude is the magnitude of the wiggles in a seismic record. The amplitude is the experession of the energy in a seismic wave as a function of space, time and frequency (Zhou, 2014). The amplitude of seismic data depends on the source, receiver, and the medium in which the waves are propagating.

In active source reflection seismology, the effect of the source and the receivers is anticipated and minimized. However, seismic attenuation due to media is typically addressed during processing. Seismic wave attenuation due to the media of propagation includes transmission loss, absorption and spherical spreading (Shearer, 2009).

Plot the raw shot gather with the plot_segy function:

In [ ]:
## Plot raw traces
startTime = 0.0
endTime = 2.5
plot_segy(stream, normalizeTraces=False, normalizeGlobal=True, 
          scale=1, title='Raw data', tstart=startTime, tend=endTime, saveFigure=False)

---
**Gain** is the amplitude correction applied to the seismic data. There are data-independent and data-dependent amplitude correction methods. In data-independent techniques, the scaling function is not derived from the amplitude information in the data.

An example of a data-independent method is the $t^{2}$ method where each trace's data is multiplied by the square of the two-way travel time (Yilmaz, 2001).

Use the gain function from the Util module to apply the $t^{2}$ gain:

---

In [ ]:
gainedStream = gain(stream, normalizeTraces=True)

Use the plot_segy function from the Util module to plot the $t^{2}$ gained dataset.

In [ ]:
startTime = 0.0
endTime = 2.5
plot_segy(gainedStream, normalizeTraces=False, normalizeGlobal=False, 
          scale=1, title='t^2 corrected', tstart=startTime, tend=endTime, saveFigure=False)

---
<font size="4.5">**4. Picking arrivals**</font><br>
<br>

![](https://raw.githubusercontent.com/unm-escape/escape2022/main/day08_Aug17/schematic.png)

Now we identify the phases and pick the arrivals.

First let's isolate one half of the raw and gained shot gather.

In [ ]:
half_offset = Stream()
for tr in stream:
    if tr.stats.segy.trace_header.distance_from_center_of_the_source_point_to_the_center_of_the_receiver_group >= 0:
        half_offset += tr
        
half_offset_gained = Stream()
for tr in gainedStream:
    if tr.stats.segy.trace_header.distance_from_center_of_the_source_point_to_the_center_of_the_receiver_group >= 0:
        half_offset_gained += tr

Pick the direct wave.

In [ ]:
## Pick first arrival
startTime = 0.0
endTime = 1.0
dw_trc_nums, dw_picks, dw_offsets = pick_segy(half_offset, normalizeTraces=True, normalizeGlobal=False, scale=1, 
                                          title=None, tstart=startTime, tend=endTime, 
                                          saveFigure=True)

---
Plot the picks on the half-offset gathers

In [ ]:
## Plot picks
%matplotlib inline
plot_segy_picks(half_offset_gained, normalizeTraces=True, normalizeGlobal=False, scale=1, title=None, 
                tstart=0, tend=1, picksX=dw_trc_nums, picksY=dw_picks, saveFigure=True)

Create a T-X plot with your picks.

In [ ]:
print(dw_offsets)
print(dw_picks)

In [ ]:
#m, b = np.polyfit(offsets, time_picks,1)
coeffs = np.polyfit(dw_offsets, dw_picks,1)
# Create the poly1d_fn function which takes in x and returns an estimate for y
poly1d_fn = np.poly1d(coeffs)

fig, axs = plt.subplots(figsize=(4,9))
plt.axis('off')
ax = fig.add_subplot('111')

#ax.plot(offsets, time_picks, color='b', lw=1.0, zorder=1)
#plt.scatter(offsets, time_picks, marker='x', color='r', s=50)

#ax.plot(offsets, time_picks, 'rx', offsets, poly1d_fn(offsets), '--k')
ax.plot(dw_offsets, dw_picks, 'rx', lw=1.0, zorder=1, label='Direct wave pick')
ax.plot(dw_offsets, poly1d_fn(dw_offsets), '--k', lw=2.0, zorder=2, alpha=0.3)

ax.set_xlabel('Distance (m)', fontsize=15)
ax.set_ylabel('TWTT (s)', fontsize=15)
#ax.text(offsets[round(len(offsets)/2)], time_picks[round(len(time_picks)/2)], str(round(1/m,2))+' m/s', fontsize=15)
plt.title('T-X plot', fontsize=20)
#plt.gca().invert_yaxis()
plt.legend()
plt.show()

Use Numpy's polyfit function to calculate $V_{0}$

In [ ]:
# y = mx + b
# y = slope*x + y-intercept
m, b = np.polyfit(dw_offsets, dw_picks, 1)
#print(1/m)
#print(b)
print('V\N{SUBSCRIPT ZERO} = ' + str(round(1/m, 2)) + ' m/s')

Use the pick_segy function to pick reflected wave

In [ ]:
## Pick reflected wave
startTime = 0.0
endTime = 1.0
rw_trc_nums, rw_picks, rw_offsets = pick_segy(half_offset, normalizeTraces=True, normalizeGlobal=False, scale=1, 
                                          title=None, tstart=startTime, tend=endTime, 
                                          saveFigure=False)

Plot reflected wave picks on the half-offset gathers

In [ ]:
## Plot picks
%matplotlib inline
plot_segy_picks(half_offset_gained, normalizeTraces=True, normalizeGlobal=False, scale=1, title=None, 
                tstart=0, tend=1, picksX=rw_trc_nums, picksY=rw_picks, saveFigure=False)

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(figsize=(4,9))
plt.axis('off')
ax = fig.add_subplot('111')

ax.plot(dw_offsets, dw_picks, 'rx', lw=1.0, zorder=1, label='Direct wave pick')
ax.plot(dw_offsets, poly1d_fn(dw_offsets), '--k', lw=2.0, zorder=2, alpha=0.3)
ax.plot(rw_offsets, rw_picks, 'bx', lw=1.0, zorder=3, label='Reflected wave pick')

ax.set_xlabel('Distance (m)', fontsize=15)
ax.set_ylabel('TWTT (s)', fontsize=15)
#ax.text(offsets[round(len(offsets)/2)], time_picks[round(len(time_picks)/2)], str(round(1/m,2))+' m/s', fontsize=15)
plt.title('T-X plot', fontsize=20)
plt.legend()
#plt.gca().invert_yaxis()
plt.show()

Create a $T^{2}$-$X^{2}$ plot of reflected picks.

In [ ]:
t_squared = np.square(rw_picks)
x_squared = np.square(rw_offsets)
#m, b = np.polyfit(offsets, time_picks,1)
rw_coeffs = np.polyfit(x_squared, t_squared, 1)
# Create the poly1d_fn function which takes in x and returns an estimate for y
rw_poly1d_fn = np.poly1d(rw_coeffs)

%matplotlib inline
fig, axs = plt.subplots(figsize=(4,9))
plt.axis('off')
ax = fig.add_subplot('111')
ax.plot(x_squared, t_squared, 'bx', lw=1.0, zorder=1)
ax.plot(x_squared, rw_poly1d_fn(x_squared), '--k', lw=2.0, zorder=2, alpha=0.3)
ax.set_xlabel('$X^{2}$ ($m^{2}$)', fontsize=15)
ax.set_ylabel('$T^{2}$ ($s^{2}$)', fontsize=15)
plt.title('$T^{2}$-$X^{2}$ plot', fontsize=20)
plt.show()

In [ ]:
# y = mx + b
# y = slope*x + y-intercept
m, b = np.polyfit(x_squared, t_squared, 1)
stackVel = np.sqrt(1/m)
print('Stacking velocity = ' + str(round(stackVel, 2)) + ' m/s')

---
### References
* Chen, Y., Jiao, S., Ma, J., Chen, H., Zhou, Y., & Gan, S. (2015). Ground-roll noise attenuation using a simple and effective approach based on local band-limited orthogonalization. IEEE Geoscience and Remote Sensing Letters, 12(11), 2316-2320.


* Shearer, P. M. (2009). Introduction to Seismology. Cambridge University Press.


* Yarham, C., Boeniger, U., & Herrmann, F. (2006, October). Curvelet-based ground roll removal. In 2006 SEG Annual Meeting. OnePetro.


* Yilmaz, Ö. (2001). Seismic data analysis: Processing, inversion, and interpretation of seismic data, Third Ed., S. M. Doherty (Editor), Vol. I, Society of Exploration Geophysicists, Tulsa, Oklahoma.


* Zhou, Hua-Wei. Practical seismic data analysis. Cambridge University Press.
